This file computes test loss and parameter sensitivity for all models.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
device = torch.device('cuda')

In [ ]:
num_classes = 37
batch_size = 4
iter_batch = 15

In [ ]:
weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1
T = weights.transforms()
data = torchvision.datasets.OxfordIIITPet('OxfordIIITPet', transform=T, download=True)
train, val = torch.utils.data.random_split(data, [3000, 680], generator=torch.Generator().manual_seed(42))
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False) # Here we do not shuffle the train data for consistency across models
test = torchvision.datasets.OxfordIIITPet('OxfordIIITPet', transform=T, download=True, split='test')
test_loader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=True)

In [ ]:
def dropout(p):
    return nn.Sequential(nn.ReLU(), nn.Dropout(p=p))
def set_dropout(model, p):
    for name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            model.relu = dropout(p)
        set_dropout(child, p)
def make_model(model_name=''):
    model = torchvision.models.resnet50(weights='IMAGENET1K_V1')
    model.fc = torch.nn.Linear(2048, num_classes, bias=False)
    set_dropout(model, 0.1)
    model = model.to(device)
    if model_name != '':
        model.load_state_dict(torch.load(f'/content/gdrive/My Drive/{model_name}'))
    model.eval();
    return model

In [ ]:
# For each model, select the iteration with lowest validation loss

def lowest_val_loss(name):
    results = np.loadtxt(f'/content/gdrive/My Drive/{name}_curve.txt')
    val_idx = results.shape[-1]-1
    return np.argmin(results[:, val_idx])

names = ['intranosmooth','intranosmoothr1',
         'intrasmooth', 'intrasmoothr1',
         'classicnosmooth','classicnosmoothr1',
         'classicsmooth', 'classicsmoothr1',]
best = [lowest_val_loss(name) for name in names]
models = [(name, make_model(f'{name}_{epoch}.mdl')) for name, epoch in zip(names, best)]

In [ ]:
losses = []
loss_fn = torch.nn.CrossEntropyLoss()

for batch, label in test_loader:
    batch = batch.to(device)
    label = label.to(device)
    batch_loss = []
    batch_correct = []
    for name, model in models:
        output = model(batch)
        loss = loss_fn(output, label)
        batch_loss.append(float(batch.shape[0] * loss))
    losses.append(batch_loss)
loss = np.array(losses).mean(0)
print('test loss:', loss)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
n_batches = len(train_loader) / (batch_size*iter_batch)
parameters = torch.concat([param.data.flatten() for param in list(models[0][1].parameters())]).detach().to('cpu')
n_parameters = len(parameters)

for name, model in models:
    model.zero_grad()
    sensitivity = torch.zeros(n_parameters)
    norm_sensitivity = torch.zeros(n_parameters)
    cumulative_loss = 0
    for i, (batch, label) in enumerate(train_loader):
        batch = batch.to(device)
        label = label.to(device)
        
        output = model(batch)
        loss = loss_fn(output, label)/iter_batch
        loss.backward()
        cumulative_loss += float(loss)

        if (i+1) % iter_batch == 0:
            gradients = torch.concat([param.grad.flatten() for param in list(model.parameters())]).detach().to('cpu')
            batch_sensitivity = torch.abs(parameters * gradients)
            norm_batch_sensitivity = torch.abs(parameters * gradients / cumulative_loss)
            sensitivity += batch_sensitivity
            norm_sensitivity += norm_batch_sensitivity
    std_sensitivity = torch.std(sensitivity / n_batches)
    norm_std_sensitivity = torch.std(norm_sensitivity/ n_batches)
    print(f'{name} standard deviation of parameter sensitivity:', float(std_sensitivity))
    print(f'{name} normalized standard deviation of parameter sensitivity:', float(norm_std_sensitivity))